<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M0.532 · Pattern Recognition</p>
<p style="margin: 0; text-align:right;">Computational Engineering and Mathematics Master</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Computers, Multimedia and Telecommunications Department</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

Reference Notebook: <br>

https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/autoencoder.ipynb


## Setup

Do all the imports needed:

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model



Lets define functions to preprocess input data and to display images 

In [ ]:
def preprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """

    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 28, 28, 1))
    return array

def display_array(array):
    """
    Displays ten random images from the supplied array.
    """

    n = 10

    indices = np.random.randint(len(array), size=n)
    images = array[indices, :]

    plt.figure(figsize=(20, 4))
    for i, image in enumerate(images):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(image.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()


def display_arrays(array1, array2):
    """
    Displays ten random images from each one of the supplied arrays.
    """

    n = 10

    indices = np.random.randint(len(array1), size=n)
    images1 = array1[indices, :]
    images2 = array2[indices, :]

    plt.figure(figsize=(20, 4))
    for i, (image1, image2) in enumerate(zip(images1, images2)):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(image1.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(image2.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()

## Prepare the data

Lets load mnist data, preprocess it with the function defined above and display the data:

In [ ]:
# Since we only need images from the dataset to encode and decode, we
# won't use the labels.
(train_data, _), (test_data, _) = mnist.load_data()

# Normalize and reshape the data
train_data = preprocess(train_data)
test_data = preprocess(test_data)

# Display the train data and a version of it with added noise
display_array(train_data) # , noisy_train_data)

## Build the autoencoder

We are going to use the Functional API to build our convolutional autoencoder.

In [ ]:
input = layers.Input(shape=(28, 28, 1))

# Encoder

# convolutional layer (trainable parameters)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
# max pooling: we reduce the output of the layer to (14, 14):
x = layers.MaxPooling2D((2, 2), padding="same")(x)
# convolutional layer (trainable parameters)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
# max pooling: we reduce the output of the layer to (7, 7):
x = layers.MaxPooling2D((2, 2), padding="same")(x)


# Decoder
# the input of the decoder has a shape of (7, 7, 32)
# notice that with the Conv2DTranspose we are increasing the rows and cols:
# https://keras.io/api/layers/convolution_layers/convolution2d_transpose/
# we are creating 32 filters, with kernels 3x3 and stride 2
# the output of the layer is of size (14, 14, 32)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
# with the following Conv2D the output of the layer is of size (28, 28, 32)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
# last layer generates an output of size (28, 28, 1): the same size of the input!
x = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

In [ ]:
# lets define the Autoencoder:
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.summary()

Now we can train our autoencoder using `train_data` as both our input data
and target. Notice we are setting up the validation data using the same
format.

In [ ]:
autoencoder.fit(
    x=train_data,
    y=train_data,
    epochs=50,
    batch_size=128,
    shuffle=True,
    validation_data=(test_data, test_data),
)

Let's predict on our test dataset and display the original image together with
the prediction from our autoencoder.

Notice how the predictions are pretty close to the original images:

In [ ]:
predictions = autoencoder.predict(test_data)
display(test_data, predictions)

The autoencoder have been able to reconstruct the input images. Optional exercice: modify the autoencoder to obtain a latent space with less dimensions until the quality of the images starts to degrade

Also you can see an application of autoencoders: remove noise


https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/autoencoder.ipynb
